[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PoseIQ/BiomechPythonAI_Guide/blob/main/PoseIQ_Chapter0_C3D_ColabNotebook_UPDATED.ipynb)

# Chapter 0: Getting Started with C3D Files in Python and Colab

**Ebook**: *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author**: Dr. Hossein Mokhtarzadeh  
**Powered by**: PoseIQ™

Welcome to Chapter 0. This notebook will walk you through loading and visualizing biomechanics C3D files in Google Colab.

We’ve been developing **PoseIQ™** tools for physio, human movement, and rehab using just a camera. This ebook and its companion course on **[Udemy](https://www.udemy.com/user/hossein-mokhtarzadeh/)** are designed to teach you by doing — no prior experience with C3D, Python, or Colab needed.

📌 **Quick Start**: Just click `Runtime > Run all`, and you're good to go.  
You'll download a sample C3D dataset, install the needed tools, load and plot your first force plate signal.

🔗 [PoseIQ™ on LinkedIn](https://www.linkedin.com/company/poseiq/)  
🔗 [Demo Page](https://poseiq.com/demos)  
🔗 [poseiq.com](https://poseiq.com)


# Chapter 0: Getting Started with C3D Files in Python and Colab

**Ebook**: *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author**: Dr. Hossein Mokhtarzadeh  
**Powered by**: PoseIQ™

Welcome to Chapter 0. This notebook will walk you through loading and visualizing biomechanics C3D files in Google Colab.

We’ve been developing **PoseIQ™** tools for physio, human movement, and rehab using just a camera. This ebook and its companion course on **Udemy** are designed to teach you by doing — no prior experience with C3D, Python, or Colab needed.

📌 **Quick Start**: Just click `Runtime > Run all`, and you're good to go.  
You'll download a sample C3D dataset, install the needed tools, load and plot your first force plate signal.

🔗 [PoseIQ™ on LinkedIn](https://www.linkedin.com/company/poseiq/)  
🔗 [Demo Page](https://poseiq.com/demos)  
🔗 [poseiq.com](https://poseiq.com)


In [ ]:
!pip install ezc3d pandas numpy

In [ ]:
import os, urllib.request, zipfile, io, sys
import numpy as np
import matplotlib.pyplot as plt

def download_samples():
    url = "https://c3d.org/data/Sample01.zip"
    local = "c3d_samples.zip"
    urllib.request.urlretrieve(url, local)
    with zipfile.ZipFile(local, "r") as z:
        z.extractall("c3d_samples")
    c3d_files = [os.path.join(r, f)
                 for r, _, fs in os.walk("c3d_samples")
                 for f in fs if f.lower().endswith(".c3d")]
    if not c3d_files:
        raise FileNotFoundError("No C3D files were found after extraction.")
    return sorted(c3d_files)

def load_with_ezc3d(path):
    import ezc3d
    c3d = ezc3d.c3d(path)
    A = c3d["data"]["analogs"]                   # shape (nSub, nChan, nFrm)
    nSub, nChan, nFrm = A.shape
    A_flat = A.transpose(2, 0, 1).reshape(nFrm*nSub, nChan)
    analog_rate = float(c3d["parameters"]["ANALOG"]["RATE"]["value"][0])
    labels = []
    try:
        labels = list(c3d["parameters"]["ANALOG"]["LABELS"]["value"])
    except Exception:
        labels = [f"CH{j+1}" for j in range(A_flat.shape[1])]
    fp_map = None
    try:
        fp_map = np.array(c3d["parameters"]["FORCE_PLATFORM"]["CHANNEL"]["value"])
        # shape often (6, nPlates) with rows Fx Fy Fz Mx My Mz and one based indices
    except Exception:
        fp_map = None
    return A_flat, analog_rate, labels, fp_map

def load_with_c3d_py(path):
    import c3d
    with open(path, "rb") as h:
        r = c3d.Reader(h)
        frames = []
        for pts, analog in r.read_frames():
            # analog shape usually (nChan, samples_per_frame)
            frames.append(analog.T)  # to (samples_per_frame, nChan)
        if not frames:
            raise RuntimeError("No analog frames in file.")
        A_flat = np.vstack(frames)
        # rate = frame_rate * analog_samples_per_frame
        frame_rate = float(r.header.frame_rate)
        samples_per_frame = int(r.header.analog_samples_per_frame)
        analog_rate = frame_rate * samples_per_frame
        # labels from params if available
        try:
            labels = [s.decode("utf-8", "ignore") if isinstance(s, bytes) else str(s)
                      for s in r.get("ANALOG", "LABELS")[0]]
        except Exception:
            labels = [f"CH{j+1}" for j in range(A_flat.shape[1])]
        # force platform channel map if available
        try:
            fp = r.get("FORCE_PLATFORM", "CHANNEL")
            fp_map = np.array(fp, dtype=np.int32)
        except Exception:
            fp_map = None
    return A_flat, analog_rate, labels, fp_map

def safe_load(path):
    # try ezc3d first
    try:
        return load_with_ezc3d(path), "ezc3d"
    except Exception as e:
        msg = f"{type(e).__name__}: {e}"
        print("ezc3d failed, trying pure python reader. Reason:", msg)
        # fall back
        return load_with_c3d_py(path), "c3d"

def group_forces(A_flat, labels, fp_map):
    """
    Returns list of dicts with Fx Fy Fz per plate.
    If fp_map is missing, attempts a label based guess.
    """
    forces = []

    if fp_map is not None and fp_map.size > 0:
        # fp_map uses one based channel indices. rows Fx Fy Fz Mx My Mz
        if fp_map.ndim == 1:
            fp_map = fp_map.reshape(6, -1)
        nPlates = fp_map.shape[1]
        for p in range(nPlates):
            idx = [c-1 if c > 0 else None for c in fp_map[:, p]]
            Fx = A_flat[:, idx[0]] if idx[0] is not None else np.zeros(A_flat.shape[0])
            Fy = A_flat[:, idx[1]] if idx[1] is not None else np.zeros(A_flat.shape[0])
            Fz = A_flat[:, idx[2]] if idx[2] is not None else np.zeros(A_flat.shape[0])
            forces.append(dict(Fx=Fx, Fy=Fy, Fz=Fz))
        return forces, nPlates, "FORCE_PLATFORM mapping"
    else:
        # fallback guess by labels containing Fx Fy Fz per plate
        # very simple heuristic: look for patterns like FZ1 or FP1_FZ etc
        lab = [lbl.upper() for lbl in labels]
        def find_one(tag):
            for j, L in enumerate(lab):
                if tag in L:
                    return j
            return None
        # try up to two plates by common label conventions
        guessed = []
        for plate_id in ["1", "2"]:
            ix = find_one(f"FX{plate_id}") or find_one(f"FP{plate_id}_FX") or find_one(f"FORCE_X_{plate_id}")
            iy = find_one(f"FY{plate_id}") or find_one(f"FP{plate_id}_FY") or find_one(f"FORCE_Y_{plate_id}")
            iz = find_one(f"FZ{plate_id}") or find_one(f"FP{plate_id}_FZ") or find_one(f"FORCE_Z_{plate_id}")
            if ix is not None or iy is not None or iz is not None:
                Fx = A_flat[:, ix] if ix is not None else np.zeros(A_flat.shape[0])
                Fy = A_flat[:, iy] if iy is not None else np.zeros(A_flat.shape[0])
                Fz = A_flat[:, iz] if iz is not None else np.zeros(A_flat.shape[0])
                guessed.append(dict(Fx=Fx, Fy=Fy, Fz=Fz))
        if guessed:
            return guessed, len(guessed), "label guess"
        else:
            # last resort: treat first three channels as Fx Fy Fz of one plate
            Fx = A_flat[:, 0] if A_flat.shape[1] > 0 else np.zeros(A_flat.shape[0])
            Fy = A_flat[:, 1] if A_flat.shape[1] > 1 else np.zeros(A_flat.shape[0])
            Fz = A_flat[:, 2] if A_flat.shape[1] > 2 else np.zeros(A_flat.shape[0])
            return [dict(Fx=Fx, Fy=Fy, Fz=Fz)], 1, "first three channels"

# 1) Data
files = download_samples()
print(f"Found {len(files)} C3D file(s). Using:", files[0])

# 2) Load with fallback
((A_flat, analog_rate, labels, fp_map), backend) = safe_load(files[0])
print(f"Loaded with {backend}. Analog samples x channels:", A_flat.shape, "Rate:", analog_rate, "Hz")

# 3) Build time vector
t = np.arange(A_flat.shape[0]) / analog_rate

# 4) Extract forces per plate
forces, nPlates, source = group_forces(A_flat, labels, fp_map)
print(f"Plates detected: {nPlates} using {source}")

# 5) Plot per plate and totals
plt.figure(figsize=(10,4))
for i, f in enumerate(forces, 1):
    plt.plot(t, f["Fz"], label=f"Plate {i} Fz")
plt.title("Vertical force per plate")
plt.xlabel("Time s"); plt.ylabel("Force"); plt.grid(True); plt.legend(); plt.show()

Fx_sum = sum(f["Fx"] for f in forces)
Fy_sum = sum(f["Fy"] for f in forces)
Fz_sum = sum(f["Fz"] for f in forces)
GRF_mag = np.sqrt(Fx_sum**2 + Fy_sum**2 + Fz_sum**2)

plt.figure(figsize=(10,4))
plt.plot(t, Fz_sum, label="Total Fz")
plt.plot(t, GRF_mag, label="GRF magnitude")
plt.title("Total ground reaction force")
plt.xlabel("Time s"); plt.ylabel("Force"); plt.grid(True); plt.legend(); plt.show()


## Summary:

### Data Analysis Key Findings

*   The zip file "/content/Sample01.zip" was successfully extracted to "/content/unzipped_c3d_files".
*   Six C3D files were identified in the extracted directory.
*   Analog data, including GRF information, was successfully extracted from all six C3D files.
*   GRF data (columns starting with 'Force.F') was isolated from the analog data for each file.
*   The GRF data from all files was combined into a single DataFrame, including a 'File' column to identify the source of each data point.
*   A single plot was generated showing the 'Force.Fz' component (vertical GRF) over time for each of the six files.

### Insights or Next Steps

*   The generated plot provides a visual comparison of the vertical GRF profiles across different C3D files, which could be useful for identifying variations in gait or movement patterns.
*   Further analysis could involve calculating peak GRF values, impulse, or other relevant metrics from the extracted data.
